In [1]:
import gc
import pandas as pd
import numpy as np
from datetime import datetime

from xgboost import XGBClassifier
from sklift.models import ClassTransformation

from sklift.metrics import uplift_at_k
import matplotlib.pyplot as plt
import catboost as cb
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
from pandas import read_csv

%matplotlib inline

In [2]:
data = read_csv('dataset.csv')
y = data['response_att']
X = data.copy()

features_list = ['group', 'response_att',
 'perdelta_days_between_visits_15_30d',
 'k_var_days_between_visits_1m',
 'k_var_days_between_visits_3m',
 'k_var_days_between_visits_15d',
 'cheque_count_6m_g48',
 'response_sms',
 'cheque_count_6m_g40',
 'k_var_count_per_cheque_6m_g27',
 'k_var_disc_share_6m_g27',
 'response_viber',
 'k_var_discount_depth_1m',
 'k_var_sku_price_6m_g48',
 'cheque_count_6m_g25',
 'cheque_count_12m_g41',
 'cheque_count_6m_g41',
 'cheque_count_12m_g32',
 'sale_count_12m_g54',
 'cheque_count_6m_g32',
 'k_var_sku_price_6m_g26',
 'k_var_disc_share_6m_g24',
 'food_share_1m',
 'k_var_cheque_3m',
 'age',
 'stdev_days_between_visits_15d',
 'cheque_count_6m_g38',
 'cheque_count_3m_g57',
 'cheque_count_12m_g25',
 'months_from_register',
 'main_format',
 'food_share_15d',
 'cheque_count_12m_g48',
 'k_var_disc_share_15d_g24',
 'sale_count_12m_g49',
 'cheque_count_6m_g21',
 'crazy_purchases_goods_count_12m',
 'cheque_count_3m_g25',
 'sale_count_3m_g57',
 'k_var_sku_price_3m_g49',
 'k_var_sku_price_15d_g49',
 'k_var_disc_share_1m_g40',
 'k_var_disc_share_3m_g24',
 'k_var_sku_price_6m_g49',
 'cheque_count_6m_g42'
 'k_var_cheque_15d'
]

X = X.filter(features_list)
pd.set_option('display.max_columns', 500)
X.head(5)

,group,response_att,perdelta_days_between_visits_15_30d,k_var_days_between_visits_1m,k_var_days_between_visits_3m,k_var_days_between_visits_15d,cheque_count_6m_g48,response_sms,cheque_count_6m_g40,k_var_count_per_cheque_6m_g27,k_var_disc_share_6m_g27,response_viber,k_var_discount_depth_1m,k_var_sku_price_6m_g48,cheque_count_6m_g25,cheque_count_12m_g41,cheque_count_6m_g41,cheque_count_12m_g32,sale_count_12m_g54,cheque_count_6m_g32,k_var_sku_price_6m_g26,k_var_disc_share_6m_g24,food_share_1m,k_var_cheque_3m,age,stdev_days_between_visits_15d,cheque_count_6m_g38,cheque_count_3m_g57,cheque_count_12m_g25,months_from_register,main_format,food_share_15d,cheque_count_12m_g48,k_var_disc_share_15d_g24,sale_count_12m_g49,cheque_count_6m_g21,crazy_purchases_goods_count_12m,cheque_count_3m_g25,sale_count_3m_g57,k_var_sku_price_3m_g49,k_var_sku_price_15d_g49,k_var_disc_share_1m_g40,k_var_disc_share_3m_g24,k_var_sku_price_6m_g49
0,1,0,1.3393,0.6479,0.8240,0.4554,6.0,0.923077,5.0,0.4845,0.6366,0.071429,0.4864,0.6718,9.0,6.0,1.0,3.0,16.0,1.0,0.1300,0.8036,0.3254,1.8741,47.0,1.7078,4.0,6.0,19.0,18.0,0,0.6488,16.0,0.0,98.0,12.0,16.0,8.0,6.0,2.1596,0.0,0.0,0.83,2.0876
1,1,0,0.0000,0.0000,1.0027,0.0000,1.0,1.000000,3.0,0.0000,0.0000,0.000000,0.0000,0.0000,2.0,1.0,1.0,1.0,2.0,1.0,0.2072,0.0000,1.0000,0.9630,57.0,0.0000,1.0,0.0,2.0,4.0,1,0.0000,1.0,0.0,2.0,0.0,0.0,2.0,0.0,0.6192,0.0,0.0,0.00,0.6192
2,0,0,0.0000,0.0000,0.5559,0.0000,6.0,1.000000,7.0,0.5762,0.8560,0.250000,0.1201,0.2407,9.0,14.0,8.0,4.0,109.0,2.0,0.0000,0.6608,0.4768,0.3295,38.0,0.0000,1.0,0.0,15.0,34.0,0,0.3739,14.0,0.0,50.0,0.0,0.0,3.0,0.0,1.8758,0.0,0.0,0.00,2.5227
3,1,0,0.0000,0.0000,0.7432,0.0000,3.0,0.909091,4.0,0.3295,0.5780,0.000000,0.0000,0.1028,11.0,8.0,2.0,2.0,39.0,2.0,0.7469,0.0000,1.0000,1.4933,65.0,0.0000,5.0,0.0,25.0,40.0,1,0.0000,6.0,0.0,49.0,1.0,3.0,5.0,0.0,0.1326,0.0,0.0,0.00,3.0736
4,1,0,0.0000,0.4714,0.9980,0.4714,4.0,1.000000,8.0,0.7526,0.9058,0.100000,0.4903,0.2195,2.0,3.0,2.0,0.0,25.0,0.0,1.0926,1.0710,0.2882,0.9014,61.0,1.4142,1.0,0.0,2.0,20.0,0,0.2882,5.0,0.0,25.0,1.0,4.0,1.0,0.0,2.0293,0.0,0.0,0.00,1.4917


In [3]:
X_train = X.copy()

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [4]:
def custom_metric(answers, take_top_ratio=0.25):
    answers.sort_values(by='uplift', inplace=True, ascending=False)
    
    n_samples = int(np.ceil(answers.shape[0] * take_top_ratio))
    answers = answers.iloc[:n_samples, :]
    answers_test = answers[answers['group'] == 1]['response_att'].sum() / \
                   answers[answers['group'] == 1].shape[0]
    answers_control = answers[answers['group'] == 0]['response_att'].sum() / \
                      answers[answers['group'] == 0].shape[0]
    return (answers_test - answers_control) * 100

In [5]:
xgb_est_params_1 = {
    'max_depth':4,
    'learning_rate': 0.17, 
    'n_estimators': 100,
    'nthread':64,
    'n_gpus':1, 
    'seed':239
}

# 5 0.17 70
# 5 0.17 59
# 4 0.17 100
# 4 0.17 140
# 3 0.22 180
# 3 0.215 155
# 3 0.2 115

xgb_est_params_2 = {
    'max_depth':3,
    'learning_rate': 0.215, 
    'n_estimators': 155,
    'nthread':64,
    'n_gpus':1, 
    'seed':42
}

xgb_est_params_3 = {
    'max_depth':4,
    'learning_rate': 0.17, 
    'n_estimators': 59,
    'nthread':64,
    'n_gpus':1, 
    'seed':179
}

xgb_est_params_4 = {
    'max_depth':3,
    'learning_rate': 0.2, 
    'n_estimators': 115,
    'nthread':64,
    'n_gpus':1, 
    'seed':179
}

estimator_1 = XGBClassifier(
    **xgb_est_params_1
)

estimator_2 = XGBClassifier(
    **xgb_est_params_2
)

estimator_3 = XGBClassifier(
    **xgb_est_params_3
)

estimator_4 = XGBClassifier(
    **xgb_est_params_4
)

uplift_model_cl_tr_1 = ClassTransformation(
    estimator=estimator_1
)

uplift_model_cl_tr_2 = ClassTransformation(
    estimator=estimator_2
)

uplift_model_cl_tr_3 = ClassTransformation(
    estimator=estimator_3
)

uplift_model_cl_tr_4 = ClassTransformation(
    estimator=estimator_4
)

In [6]:
uplift_model_cl_tr_1.fit(
    X=X_train.drop(columns=['group', 'response_att']),
    y=X_train['response_att'],
    treatment=X_train['group']
)

uplift_model_cl_tr_2.fit(
    X=X_train.drop(columns=['group', 'response_att']),
    y=X_train['response_att'],
    treatment=X_train['group']
)

uplift_model_cl_tr_3.fit(
    X=X_train.drop(columns=['group', 'response_att']),
    y=X_train['response_att'],
    treatment=X_train['group']
)

uplift_model_cl_tr_4.fit(
    X=X_train.drop(columns=['group', 'response_att']),
    y=X_train['response_att'],
    treatment=X_train['group']
)

/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning:

It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.

/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning:

It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.

/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning:

It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.

/anaconda/envs/azureml_py36_automl/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning:

It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.



ClassTransformation(estimator=XGBClassifier(learning_rate=0.2, n_estimators=115,
                                            n_gpus=1, nthread=64, seed=179))

In [7]:
pred_1 = uplift_model_cl_tr_1.predict(X_test.drop(columns=['group', 'response_att']))
pred_2 = uplift_model_cl_tr_2.predict(X_test.drop(columns=['group', 'response_att']))
pred_3 = uplift_model_cl_tr_3.predict(X_test.drop(columns=['group', 'response_att']))
pred_4 = uplift_model_cl_tr_4.predict(X_test.drop(columns=['group', 'response_att']))
uplift_ts = pred_1 + pred_2 + pred_3 + pred_4

df_submit = X_test.assign(uplift=uplift_ts)[['uplift']]
df_submit['group'] = X_test['group']
df_submit['response_att'] = X_test['response_att']
print(custom_metric(df_submit))

#answer = X[['CardHolder']].assign(uplift=uplift_ts)



#answer = answer.reset_index(drop=True)
#answer.to_csv('answer2.csv', index=False, sep=';')

NameError: name 'X_test' is not defined

In [8]:
features_list = features_list[2:]
data = read_csv('test.csv')
X = data.copy()
X_test = X.filter(features_list)
pred_1 = uplift_model_cl_tr_1.predict(X_test)
pred_2 = uplift_model_cl_tr_2.predict(X_test)
pred_3 = uplift_model_cl_tr_3.predict(X_test)
uplift_ts = pred_1 + pred_2 + pred_3 / 3
answer = X[['CardHolder']].assign(uplift=uplift_ts)
answer = answer.reset_index(drop=True)
answer.to_csv('answer15.csv', index=False, sep=';')